In [1]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '1'

import logging
import numexpr as ne
import numpy as np
import torch

from ddopai.experiments.meta_experiment_functions import *
import requests
import yaml
import re
import pandas as pd
import wandb
from copy import deepcopy
import warnings
import gc
from mushroom_rl import core 
import pickle
from tqdm import tqdm, trange

In [2]:

config_env = import_config("config_env.yaml")

INFO:root:Configuration file 'config_env.yaml' successfully loaded.


In [3]:
env_kwargs = []
for episode in range(config_env["setup_kwargs"]["n_episodes"]):
    episode_kwargs = {}
    episode_kwargs['env_class'] = config_env["setup_kwargs"]["env_class"][episode % len(config_env["setup_kwargs"]["env_class"])]
    episode_kwargs['horizon_train'] = config_env["setup_kwargs"]["horizon_train"][episode % len(config_env["setup_kwargs"]["horizon_train"])]
    episode_kwargs['p_bound_low'] = config_env["setup_kwargs"]["p_bound_low"][episode % len(config_env["setup_kwargs"]["p_bound_low"])]
    episode_kwargs['p_bound_high'] = config_env["setup_kwargs"]["p_bound_high"][episode % len(config_env["setup_kwargs"]["p_bound_high"])]
    episode_kwargs['inv'] = config_env["setup_kwargs"]["inv"][episode % len(config_env["setup_kwargs"]["inv"])]
    episode_kwargs['nb_features'] = config_env["setup_kwargs"]["nb_features"][episode % len(config_env["setup_kwargs"]["nb_features"])]
    episode_kwargs['covariance'] = config_env["setup_kwargs"]["covariance"][episode % len(config_env["setup_kwargs"]["covariance"])]
    episode_kwargs['noise_std'] = config_env["setup_kwargs"]["noise_std"][episode % len(config_env["setup_kwargs"]["noise_std"])]
    episode_kwargs['function_form'] = config_env["setup_kwargs"]["function_form"][episode % len(config_env["setup_kwargs"]["function_form"])]
    episode_kwargs['env_type'] = config_env["setup_kwargs"]["env_type"][episode % len(config_env["setup_kwargs"]["env_type"])]
    alpha = np.random.multivariate_normal(
        mean=np.zeros(episode_kwargs['nb_features'] - 1),
        cov=np.eye(episode_kwargs['nb_features'] - 1) * config_env["setup_kwargs"]["covariance_parameter"][episode % len(config_env["setup_kwargs"]["covariance_parameter"])]
    )
    beta = np.random.multivariate_normal(
        mean=np.zeros(episode_kwargs['nb_features'] - 1),
        cov=np.eye(episode_kwargs['nb_features'] - 1) * config_env["setup_kwargs"]["covariance_parameter"][episode % len(config_env["setup_kwargs"]["covariance_parameter"])]
    )
    sparsity = config_env["setup_kwargs"]["sparsity_parameter"][episode % len(config_env["setup_kwargs"]["sparsity_parameter"])]
    num_zero_alpha = int(len(alpha) * sparsity)
    num_zero_beta = int(len(beta) * sparsity)

    zero_indices_alpha = np.random.choice(len(alpha), num_zero_alpha, replace=False)
    zero_indices_beta = np.random.choice(len(beta), num_zero_beta, replace=False)

    alpha[zero_indices_alpha] = 0
    beta[zero_indices_beta] = 0
    
    alpha = np.insert(alpha, 0, config_env["setup_kwargs"]["leading_alpha"][episode % len(config_env["setup_kwargs"]["leading_alpha"])])
    beta = np.insert(beta, 0, config_env["setup_kwargs"]["leading_beta"][episode % len(config_env["setup_kwargs"]["leading_beta"])])
    
    episode_kwargs['alpha'] = [round(float(a), 2) for a in alpha]
    episode_kwargs['beta'] = [round(float(b), 2) for b in beta]
    
    env_kwargs.append(episode_kwargs)
config_env["env_kwargs"] = env_kwargs

In [4]:
env_kwargs[0]

{'env_class': 'DynamicPricingEnv',
 'horizon_train': 400,
 'p_bound_low': 0,
 'p_bound_high': 20,
 'inv': [[[30000.0]]],
 'nb_features': 10,
 'covariance': [1.5],
 'noise_std': [0.5],
 'function_form': ['linear'],
 'env_type': {'inv': False, 'reference_price': False},
 'alpha': [10.0, 0.39, 0.05, -0.02, -0.19, -0.03, 0.34, 0.02, -0.12, 0.3],
 'beta': [-1.0, 0.17, -0.01, 0.02, -0.06, -0.2, 0.17, 0.05, -0.23, -0.17]}

In [5]:
class NoAliasDumper(yaml.SafeDumper):
    def ignore_aliases(self, data):
        return True

with open("config_env.yaml", "w") as file:
    yaml.dump(config_env, file, default_flow_style=False, Dumper=NoAliasDumper)